In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.
import torchvision.models as models
import matplotlib.pyplot as plt
import numpy as np
import os
import glob as gb
import cv2
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from torchsummary import summary



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Hyperparameters
in_channels = 3
num_classes = 4
learning_rate = 0.001
batch_size = 64
# Load Data
trainpath ='/content/drive/MyDrive/covid19/pytorch3/train/'
testpath='/content/drive/MyDrive/covid19/pytorch3/test/'
valpath='/content/drive/MyDrive/covid19/pytorch3/val/'

img_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize(size=(224,224)), 
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]), 
        ]),

    'test':
    transforms.Compose([
        transforms.Resize(size=(224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]), 
        ]),
        'val':
    transforms.Compose([
        transforms.Resize(size=(224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]), 
        ]),
     }

train_dataset=datasets.ImageFolder(trainpath,transform=img_transforms['train'])
test_dataset=datasets.ImageFolder(testpath,transform=img_transforms['test'])
val_dataset=datasets.ImageFolder(testpath,transform=img_transforms['val'])
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True)


it=iter(train_loader)
images,label=it.next()
print(images.shape)
print(label.shape)



torch.Size([64, 3, 224, 224])
torch.Size([64])


In [5]:
def train(num_epochs,model,train_loader):

# Loss and optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters())
  trainingloss = []
  validationloss = []
# Train the model
  total_step = len(train_loader)
  for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
              
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()  
    with torch.no_grad():
       correct = 0
       total = 0
       for images, labels in val_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    acc=(100 * correct / total)    
          # printing training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch,acc))

In [6]:
# Test the model
def test(model,test_loader): 

  model.eval()  
  with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

  print('Test Accuracy of the model is: {} %'.format(100 * correct / total))


In [4]:
!pip install --upgrade pytorch-pretrained-vit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pytorch-pretrained-vit: filename=pytorch_pretrained_vit-0.0.7-py3-none-any.whl size=11132 sha256=008cdf4003c5f5a861072c57f5f5e359dfc2f0047c5df9120d69d2f5c27d17ac
  Stored in directory: /root/.cache/pip/wheels/87/1d/d1/c6852ef6d18565e5aee866432ab40c6ffbd3411d592035cddb
Successfully built pytorch-pretrained-vit


In [5]:
from pytorch_pretrained_vit import ViT

In [15]:
!pip install timm

import timm

class ViTBase16(nn.Module):
    def __init__(self, n_classes, pretrained=False):

        super(ViTBase16, self).__init__()

        self.model = timm.create_model("vit_base_patch16_224", pretrained=False)

        self.model.head = nn.Linear(self.model.head.in_features, n_classes)

    def forward(self, x):
        x = self.model(x)
        return x

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
model = ViTBase16(n_classes=4, pretrained=True)


In [ ]:
num_epochs=50

train(num_epochs,model,train_loader)
#session crached every time i run this code

In [ ]:
test(model,test_loader)